### Purpose of the Dataset

This data set is from the well-known UCI Machine Learning Repository. The objective of this notebook is to correctly forecast, based on the provided features, whether or not an adult earns more than $50,000 USD per year.


### About the Dataset

This data set is from the well-known UCI Machine Learning Repository. The objective of this notebook is to correctly forecast, based on the provided features, whether or not an adult earns more than $50,000 USD per year.

The data set selected is “Adult Census Income” attached is the link for easy reference – https://archive.ics.uci.edu/ml/datasets/Adult

Age: Describes the age of individuals. Continuous.

Workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

fnlwgt: Continuous.

education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. education-num: Number of years spent in education. Continuous.

marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

sex: Female, Male.

capital-gain: Continuous.

capital-loss: Continuous.

hours-per-week: Continuous.

native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

salary: >50K,<=50K

### Step1: Import all required libraries

In [2]:
import pandas as pd
from sklearn import *
from sklearn import preprocessing 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import plot_tree
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import warnings 
warnings.filterwarnings('ignore')
from __future__ import print_function
from sklearn.neural_network import MLPClassifier
import os
import tensorflow as tf2
from tensorflow import keras
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from sklearn import datasets
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt

from sklearn import datasets
np.random.seed(1)

### Step2: Load the data, clean and prepare data for analysis

In [5]:
final_df = pd.read_csv("census_final_df.csv") # change this path depending on where you store the file on your computer

In [6]:

final_df.head(5)

,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION-NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,NATIVE_COUNTRY,INCOME
0,39,6,77516,9,13,4,0,1,4,1,2174,0,40,38,0
1,50,5,83311,9,13,2,3,0,4,1,0,0,13,38,0
2,38,3,215646,11,9,0,5,1,4,1,0,0,40,38,0
3,53,3,234721,1,7,2,5,0,2,1,0,0,40,38,0
4,28,3,338409,9,13,2,9,5,2,0,0,0,40,4,0


### Step3: Split Data

Split data into a 70/30 split.

In [7]:
train_df, test_df = train_test_split(final_df, test_size=0.3)
target = 'INCOME'
predictors = ['AGE','EDUCATION','EDUCATION-NUM','MARITAL_STATUS','OCCUPATION','RELATIONSHIP','RACE','SEX','CAPITAL_GAIN','CAPITAL_LOSS','HOURS_PER_WEEK']



In [8]:
train_X = train_df[predictors]
train_y = train_df[target]
test_X = test_df[predictors]
test_y = test_df[target]

## 4. Model the data

First, we will create a dataframe to hold all the results of our models.

In [9]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

###  Fit and test a Logistic Regression model

### LOGISTIC REGRESSION MODEL- PARAMETER TUNING USING RANDSOM SEARCH

In [7]:
score_measure = "recall"
kfolds = 5
space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']



log_reg_model = LogisticRegression()
rand_search = RandomizedSearchCV(log_reg_model, space, n_iter=500, scoring='recall', n_jobs=-1, cv=kfolds, random_state=1)
_ = rand_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecall = rand_search.best_estimator_

The best recall score is 0.7872322823075679
... with parameters: {'solver': 'lbfgs', 'penalty': 'none'}


In [8]:
c_matrix = confusion_matrix(test_y, rand_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Random search_logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
#print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

### LOGISTIC REGRESSION MODEL- PARAMETER TUNING USING GRID SEARCH 

Conduct an exhaustive search across a smaller range of parameters around the parameters found in the initial random search.

In [9]:
score_measure = "recall"
kfolds = 2

param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet'],
    'C' : np.logspace(-4, 4),
    'solver' : ['lbfgs','newton-cg','liblinear'],
    'max_iter' : [100,200, 1000,]
    }
]

log_reg_model = LogisticRegression()
grid_search = GridSearchCV(estimator = log_reg_model, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecall = grid_search.best_estimator_

Fitting 2 folds for each of 1350 candidates, totalling 2700 fits
The best recall score is 0.858883015398759
... with parameters: {'C': 0.0009540954763499944, 'max_iter': 200, 'penalty': 'l1', 'solver': 'liblinear'}


In [10]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Grid search_logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Random search_logistic,0.736246,0.706604,0.795653,0.748489
0,Grid search_logistic,0.736988,0.704173,0.804948,0.751196


### SVM MODEL-PARAMETER TUNING USING RANDSOM SEARCH

In [11]:
score_measure = "recall"
kfolds = 5

svm_param = {
    "C": [.01, .1, 1,],
    "gamma": [.01, .1, 1,],
    "kernel": ["rbf"],
    "random_state": [1]
}

svm_lin_model = SVC()
rand_search = RandomizedSearchCV(estimator = svm_lin_model, param_distributions=svm_param, cv=kfolds, n_iter=200,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecall = rand_search.best_estimator_

Fitting 5 folds for each of 9 candidates, totalling 45 fits
The best recall score is 1.0
... with parameters: {'random_state': 1, 'kernel': 'rbf', 'gamma': 1, 'C': 0.01}


In [12]:
c_matrix = confusion_matrix(test_y, rand_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Random search_SVC", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
#print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

### SVM MODEL- PARAMETER TUNING USING GRID SEARCH

Conduct an exhaustive search across a smaller range of parameters around the parameters found in the initial random search.

In [13]:
score_measure = "recall"
kfolds = 5

param_grid = {'C': [0.1, 1], 
              'gamma': [2,1, 0.1],
              'kernel': ['rbf']}

svm_lin_model = SVC()

grid_search = GridSearchCV(estimator = svm_lin_model, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecall = grid_search.best_estimator_

Fitting 5 folds for each of 6 candidates, totalling 30 fits
The best recall score is 0.9913814401331364
... with parameters: {'C': 0.1, 'gamma': 2, 'kernel': 'rbf'}


In [14]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Grid search_SVC", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Random search_logistic,0.736246,0.706604,0.795653,0.748489
0,Grid search_logistic,0.736988,0.704173,0.804948,0.751196
0,Random search_SVC,0.493258,0.493258,1.000000,0.660647
0,Grid search_SVC,0.682309,0.612145,0.971432,0.751030


### DECISION TREE CLASSIFIER

In [15]:
from sklearn.tree import DecisionTreeClassifier
dtree=DecisionTreeClassifier(random_state=1)

In [16]:
dtree.fit(train_X, train_y)
dtree_prediction_output = dtree.predict(test_X)
dtree_prediction_output[:100]

array([1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1], dtype=int64)

In [17]:
unique, counts = np.unique(dtree_prediction_output, return_counts=True)
dict(zip(unique, counts))

{0: 6813, 1: 8019}

The above output means that we found 8019 of the 14832 observations in the validation set to be likely to have income more than 50K.

#### CONFUSION MATRIX - DECISION TREE CLASSIFIER

In [18]:
confusion = confusion_matrix(test_y, dtree_prediction_output)

In [19]:
confusion

array([[6322, 1194],
       [ 491, 6825]], dtype=int64)

**TRUE POSITIVE:** Adults whose salary>50K and model actually predicted as salary>50K$ ie 6825

**TRUE NEGATIVE:** Adults whose salary>50K but model not predicted as salary>50K$ ie 6322

**FALSE POSITIVE:** Adults whose salary is not greater than 50K and model actually predicted as salary>50K$ ie 1194

**FALSE NEGATIVE:** Adults whose salary>50K and model actually predicted as not-greater than 50k$ ie 491

### DECISION TREE-PARAMETER TUNING USING RANDSOM SEARCH

In [20]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,10),  
    'min_samples_leaf': np.arange(1,10),
    'min_impurity_decrease': np.arange(0.0001, 0.0002),
    'max_leaf_nodes': np.arange(5, 10), 
    'max_depth': np.arange(1,5), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecall = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.9274885503240909
... with parameters: {'min_samples_split': 6, 'min_samples_leaf': 3, 'min_impurity_decrease': 0.0001, 'max_leaf_nodes': 8, 'max_depth': 2, 'criterion': 'entropy'}


In [21]:
c_matrix = confusion_matrix(test_y, rand_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Random search_dtree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
#print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

### DECISION TREE-PARAMETER TUNING USING GRID SEARCH

Conduct an exhaustive search across a smaller range of parameters around the parameters found in the initial random search.

In [22]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(4,8),  
    'min_samples_leaf': np.arange(1,3),
    'min_impurity_decrease': np.arange(0.0001, 0.0002),
    'max_leaf_nodes': np.arange(5, 10), 
    'max_depth': np.arange(2,4), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecall = grid_search.best_estimator_

Fitting 5 folds for each of 160 candidates, totalling 800 fits
The best recall score is 0.9274885503240909
... with parameters: {'criterion': 'entropy', 'max_depth': 2, 'max_leaf_nodes': 5, 'min_impurity_decrease': 0.0001, 'min_samples_leaf': 1, 'min_samples_split': 4}


In [23]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Grid search_dtree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Random search_logistic,0.736246,0.706604,0.795653,0.748489
0,Grid search_logistic,0.736988,0.704173,0.804948,0.751196
0,Random search_SVC,0.493258,0.493258,1.000000,0.660647
0,Grid search_SVC,0.682309,0.612145,0.971432,0.751030
0,Random search_dtree,0.719188,0.648143,0.942182,0.767979
0,Grid search_dtree,0.719188,0.648143,0.942182,0.767979


## 5.0 Summary

Sorted by RECALL, the best models are:

In [26]:
performance.sort_values(by=['Recall'])

,model,Accuracy,Precision,Recall,F1
0,Random search_logistic,0.736246,0.706604,0.795653,0.748489
0,Grid search_logistic,0.736988,0.704173,0.804948,0.751196
0,Random search_dtree,0.719188,0.648143,0.942182,0.767979
0,Grid search_dtree,0.719188,0.648143,0.942182,0.767979
0,Grid search_SVC,0.682309,0.612145,0.971432,0.751030
0,Random search_SVC,0.493258,0.493258,1.000000,0.660647


### Neural Net

In [7]:
%%time

ann = MLPClassifier(hidden_layer_sizes=(60,50,40), solver='adam', max_iter=200)
_ = ann.fit(train_X, train_y)

CPU times: total: 2min 58s
Wall time: 1min 2s


In [8]:
%%time
y_pred = ann.predict(test_X)

CPU times: total: 15.6 ms
Wall time: 41 ms


In [9]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.75      0.81      7516
           1       0.77      0.90      0.83      7316

    accuracy                           0.82     14832
   macro avg       0.83      0.82      0.82     14832
weighted avg       0.83      0.82      0.82     14832



## MLP CLASSIFIER- With RandomizedSearchCV

In [7]:
%%time

score_measure = "recall"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (50,), (70,),(50,30), (40,20), (60,40, 20), (70,50,40)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0, .2, .5, .7, 1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1, 0.2, 0.5],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = RandomizedSearchCV(estimator = ann, param_distributions=param_grid, cv=kfolds, n_iter=100,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'solver': 'sgd', 'max_iter': 5000, 'learning_rate_init': 0.01, 'learning_rate': 'invscaling', 'hidden_layer_sizes': (60, 40, 20), 'alpha': 0.7, 'activation': 'logistic'}
CPU times: total: 37.6 s
Wall time: 1h 19min 25s


In [8]:
%%time
y_pred = bestRecallTree.predict(test_X)

print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      7516
           1       0.49      1.00      0.66      7316

    accuracy                           0.49     14832
   macro avg       0.25      0.50      0.33     14832
weighted avg       0.24      0.49      0.33     14832

CPU times: total: 93.8 ms
Wall time: 124 ms


## MLP CLASSIFIER -With GridSearchCV

In [13]:
%%time

score_measure = "recall"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (30,), (70,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [.5, 1],
    'learning_rate': ['adaptive', 'invscaling'],
    'learning_rate_init': [ 0.01, 0.15],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = GridSearchCV(estimator = ann, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
{'activation': 'tanh', 'alpha': 0.5, 'hidden_layer_sizes': (70,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.01, 'max_iter': 5000, 'solver': 'adam'}
CPU times: total: 6.8 s
Wall time: 2min 6s


In [14]:
%%time
y_pred = bestRecallTree.predict(test_X)

print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.67      0.77      7516
           1       0.73      0.93      0.82      7316

    accuracy                           0.79     14832
   macro avg       0.82      0.80      0.79     14832
weighted avg       0.82      0.79      0.79     14832

CPU times: total: 62.5 ms
Wall time: 65.5 ms


### Summary of Evaluation Metrics for the models

### Metrics Considered

### Results
### 1. SVM Tuning Model predicted (97.54%-94.21%) = 3.33% better than Decision Tree Tuning model, but the recall score of random search in SVM model gives us 100% that tell us SVM tuning model is overfitting the data and Logistic regression tuning model predicted 80% of adults having greater than 50k$

### 2. When Decision Tree Tuning Model predicted salary greater than 50K$, Dtree is correct 94.2% of the time using Grid Search , where as Logistic regression Tuning Model is correct only 80% time.






 As there is a significant huge amount of False Positives we are considering recall as evaluation metrics.

3. Based on the above values of recall we can say that the Hyperparameter tuned dtree model outperformed the Tuned Logistic and svm model.


**Recall:** Interms of Recall, decision tree Tuned Model performed better than logistic regression  model and there is an overfitting in svm model

### when we compare the results of the evaluation metrics of logistic, SVM and Decision Tree with Neural Networks we got the recall score of 0.84 where as the recall score of decision tree was 0.94~ 94%
### The recall score of Neural Networks (84%) is more than the recall score of Logistic regression (80%) 